# Part 1: Scraping the data

The first step of this project was scraping all the data that could be found within the DTU course database. This is very similar to how we scraped the data involving the philosopher pages from wikipedia in the course. We had to, however, circumvent some security mechanism (automated web browser re-direction) which lead to the use of selenium webdriver. Then the scraped data was structured so they could be exported as csv files. 

In [4]:
# THE CODE THAT DID THE MAGIC! BUT REMEMBER TO INSTALL SELENIUM AND WEBDRIVER, DO SOME BROWSING TO FIGURE IT OUT. ALSO ENDEAVOUR
# RUN THE CELL WITH ONLY ONLY LIBRARIES.:)

import urllib2
from bs4 import BeautifulSoup
from lxml import html
import xml.etree.ElementTree as ET
from selenium import webdriver
import unicodecsv as csv
from io import BytesIO



depts = {"1":u"Department of Applied Mathematics and Computer Science","10":u"Department of Physics","11": 
         u"Department of Civil Engineering", "12":u"Department of Environmental Engineering","13":u"Department of Transport",
        "23":u"National Food Institute", "24":u"National Veterinary Institute", "25": u"National Institute of Aquatic Resources",
         "26":u"Department of Chemistry","27":u"Department of Systems Biology", "28":u"Department of Chemical Engineering", "30":
        u"National Space Institute", "31":u"Department of Electrical Engineering", "33": u"Department of Micro and Nanotechnology",
        "34": u"Department of Photonics Engineering","41": u"Department of Mechanical Engineering","42":
         u"DTU Management Engineering","46": u"DTU Wind Energy", "47": u"DTU", "83": u"Other courses", "IHK":u"DTU Diplom"}

for bros in depts:
    
    columns = [u"COURSE NUMBER",
               u"COURSE NAME",
               u"LANGUAGE OF INSTRUCTION",
               u"CREDIT LOAD",
               u"YEAR",
               u"SCHEDULE",
               u"COURSE TYPE",
               u"DEPARTMENT",
               u"GENERAL COURSE OBJECTIVES",
               u"LEARNING OBJECTIVES",
               u"CONTENT",
               u"RECOMMENDED PREREQUISITES",
               u"COURSE RESPONSIBLE",
               ]
    
    
    with open(depts[bros]+u'.csv', 'w') as csvfile:
        fieldnames = columns
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    

    baseurl = "http://kurser.dtu.dk/search?CourseCode=&SearchKeyword=&SchedulePlacement=&Department=%s&CourseType=&TeachingLanguage=" %bros
    
    
    buks = getit(baseurl)
    
    buks = BeautifulSoup(buks,"lxml")
    b = list()
    fin = []
    
    
    for row in buks('table', {'class': 'table'})[1].tbody('tr'):
        b.append(list(row('td')))
        
    b = b[1:]
    
    for pops in b:
        if len(pops)>=4:
            fin.append(pops)
        else:
            break
            
        
    for ro in range(len(fin)):
        v = []
        fil = filter(None,fin[ro][1].get_text().split("\n"))#[3]#.split("-")[1]#.split("|")
        v.append(fil[0].split("-")[0])
        v.append(fil[0].split("-")[1])
        v.append(fil[1].split("|")[0])
        v.append(fil[1].split("|")[1])
        v.append(fil[1].split("|")[2])
        v.append(' '.join(fil[2:]))
        v.append(','.join(fin[ro][2].get_text().split()))
        v.append(depts[bros])
        
        
        
        urlsec = "http://kurser.dtu.dk/course/%s" %v[0]
        twobuks = getit(urlsec)
        twobuks = BeautifulSoup(twobuks,"lxml")
        gg = None
        hh = None
        ii = None
        tempor = []
        stanlo = twobuks.find_all("div",class_="box")
        
        
        for tf in range(len(list(stanlo[1]))):
            try:
                if list(stanlo[1])[tf].text=="Learning objectives":
                    gg = tf
            except:
                pass
        for ie in list(stanlo[1])[1:gg]:
            try:
                tempor.append(ie.get_text(' '))
            except:
                tempor.append(ie)
        v.append(filter(None,''.join(tempor)))
        
        tempor = []
        gg = list(stanlo[1])[gg+2:]
        for tr in range(len(gg)):
            try:
                if gg[tr].text=="Content":
                    hh = tr
            except:
                pass
            
        for eg in gg[0:hh]:
            try:
                tempor.append(eg.get_text(' '))
            except:
                tempor.append(eg)
        v.append(filter(None,''.join(tempor)))
        
        hh = gg[hh+1:]
        for li in range(len(hh)):
            try:
                if hh[li].name == 'div':
                    ii = li
                    break
            except:
                pass
        
        tempor = []

        for cj in hh[:ii]:
            try:
                tempor.append(cj.get_text(' '))
            except:
                tempor.append(cj)
        v.append(filter(None,''.join(tempor)))
        
        gg = None
        hh = None
        tempor = []
        
        stanlo = twobuks.find_all("table")
        gg = stanlo[1].find_all("tr")
        
        for i in range(len(gg)):
            if "Recommended prerequisites" in gg[i].get_text():
                hh = ' '.join([k.text for k in list(gg[i])[1:]])
                hh = re.sub(ur'[\W_]+', u' ', hh, flags=re.UNICODE)
                hh = [ij for ij in hh.split() if  len(ij) == 5]
                hh = u','.join(set([ip for ip in hh if ip.isalpha()==False]))
                v.append(hh)
                
            else:
                tempor.append('None')
                
        if len(tempor)==len(gg):
            v.append(u'None')
        
        
        tempor = []
        gg = stanlo[2].find_all("tr")
        
        for crew in range(0,len(gg[0].find_all("a")),2):
            it = gg[0].find_all("a")[crew].text
            tempor.append(it)
            
        v.append(u','.join(tempor))
        

        with open(depts[bros]+".csv",'a') as b:
            wr = csv.writer(b,dialect='excel',encoding='utf-8')
            wr.writerow(v)

WebDriverException: Message: 'ChromeDriver executable needs to be available in the path.                 Please download from http://chromedriver.storage.googleapis.com/index.html                and read up at http://code.google.com/p/selenium/wiki/ChromeDriver' 

In [ ]:
gg = []
ebuks = BeautifulSoup(buks,"lxml")
#for row in ebuks('table', {'class': 'table'})[0].tbody('tr'):
#        gg.append(list(row('td')))
        
stanlo = ebuks.find_all("div",class_="box")
#for ij in range(len(list(stanlo[1]))):
#    try:
#        if list(stanlo[1])[ij].text=="Learning objectives":
#            print ij
#    except:
#        pass
    
#yi = []
#for tf in list(stanlo[1])[1:19]:
#    try:
#        yi.append(tf.text)
#    except:
#        yi.append(tf)
#filter(None,''.join(yi))
#print list(stanlo[1])[6]
i = []
i.append(list(stanlo[1])[6])#.get_text(' ')
print i

In [592]:
while True:
    browser = webdriver.Chrome()
    browser.get("http://kurser.dtu.dk/course/62L14")
    buks = browser.page_source
    if len(browser.page_source)>6000:
        #buks = browser.page_source
        print buks
        browser.quit()
        break
    else:
        browser.quit()

<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" class=" js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths"><head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Course Base</title>
    <link href="/bundles_css_general?v=qVqqVIQy6avPbLKKmHYjFX6_q13zG0NuKo5df4PFThw1" rel="stylesheet" />

    <script src="/bundles_js_general?v=-0apSkCWt1Bhbe8xDX8ezCAlskhkkWCFtEY3HLUdcEk1"></script>

    


    <script>
        $(function () {
            $("[dat

In [593]:
ss = ebuks.find_all("table")
gg = ss[1].find_all("tr")
aq = []
for i in range(len(gg)):
    if "Recommended prerequisites" in gg[i].get_text():
        t = ' '.join([k.text for k in list(gg[i])[1:]])
        t = re.sub(ur'[\W_]+', u' ', t, flags=re.UNICODE)
        t = [ij for ij in t.split() if  len(ij) == 5]
        t = ','.join(set([ip for ip in t if ip.isalpha()==False]))
print t
#gg

In [605]:
ff = ss[2].find_all("tr")
ff[0].find_all("a")[0].text

u'\nPer Goltermann'

In [663]:
df = pd.read_csv('Department of Photonics Engineering.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,COURSE NUMBER,COURSE NAME,LANGUAGE OF INSTRUCTION,CREDIT LOAD,YEAR,SCHEDULE,COURSE TYPE,DEPARTMENT,GENERAL COURSE OBJECTIVES,LEARNING OBJECTIVES,CONTENT,RECOMMENDED PREREQUISITES,COURSE RESPONSIBLE
1,34020,Optics and Photonics,English,5 ECTS,2016/2017,F1B (Thurs 13-17),BSc,Department of Photonics Engineering,\r\nThe course provides a broad introduction t...,"Identify whether a ray optics, a scalar wave o...","\r\nThe following concepts and phenomena, and ...","10013,31400,10036,10009",\r\nMartijn Wubs
2,34021,Introduction to Optics and Photonics,Danish,5 ECTS,2016/2017,F1B (Thurs 13-17) or E2B (Thurs 8-12),BSc,Department of Photonics Engineering,\r\nThe course provides a broad and fundamenta...,Calculate and measure the radiometric and phot...,"\r\nThe following concepts and phenomena, and ...","31400,10036,34120,10020",\r\nMichael Linde Jakobsen
3,34029,Physics Project,Danish,10 ECTS,2016/2017,Fall and January or Spring and June,BSc,Department of Photonics Engineering,\r\nTo perform a larger experimental or theore...,find and select relevant literature search out...,\r\nProject: Subjects with relation to the res...,10030,\r\nMorten Bache
4,34031,Applied Mathematics for Physicists,English,5 ECTS,2016/2017,F3A (Tues 8-12),BSc,Department of Photonics Engineering,"\r\nTo introduce concepts, tools and methods f...",determine whether an equation is linear or non...,\r\nFourier- and Laplace integral transformati...,"01005,10020,10036",\r\nOle Bang
5,34032,Optical Properties of Solids,English,5 ECTS,2016/2017,E1B (Thurs 13-17),MSc,Department of Photonics Engineering,"\r\nThis course, formerly called Modern Photon...",Derive the Kramers-Kronig relations for linear...,\r\nElectromagnetism in dielectrics and metals...,"34033,10036,10303,10102","\r\nMartijn Wubs,\r\nNicolas Stenger"
6,34034,Applied photonics,English,5 ECTS,2016/2017,E1A (Mon 8-12),MSc,Department of Photonics Engineering,\r\nThe course aims towards an introduction to...,design a resonator system with special propert...,"\r\nLaser beams, laser resonators, photonics, ...","34020,34127,34033,10370","\r\nPeter Tidemand-Lichtenberg,\r\nKresten Yvind"
7,34041,Waveguide optics,English,5 ECTS,2016/2017,F4B (Fri 8-12),MSc,Department of Photonics Engineering,\r\nTo establish a solid understanding of the ...,Describe the structure of modal fields in in o...,"\r\nStarting from Maxwell's equations, the fun...","31400,10036,34033,10370","\r\nJesper Lægsgaard,\r\nPeter Uhd Jepsen"
8,34042,Quantum Photonic Communication,English,5 ECTS,2016/2017,E3A (Tues 8-12),MSc,Department of Photonics Engineering,\r\nQuantum photonics is now ready to be trans...,Describe the concept and the advantages of qua...,\r\nThe course will be composed of:\r\n\r\n• L...,None,\r\nNika Akopian
9,34046,Experimental Methods in Photonics,English,5 ECTS,2016/2017,January,MSc,Department of Photonics Engineering,\r\nThe focus of this course is to introduce s...,Operate freely and explain differences between...,\r\nThe course contains lectures on\r\n\r\n• R...,34020,"\r\nPeter Uhd Jepsen,\r\nKrzysztof Iwaszczuk"


In [ ]:
import urllib2
from bs4 import BeautifulSoup
from lxml import html
import xml.etree.ElementTree as ET
from selenium import webdriver




depts = {"1":"Department of Applied Mathematics and Computer Science","10":"Department of Physics","11": 
         "Department of Civil Engineering", "12":"Department of Environmental Engineering","13":"Department of Transport",
        "23":"National Food Institute", "24":"National Veterinary Institute", "25": "National Institute of Aquatic Resources",
         "26":"Department of Chemistry","27":"Department of Systems Biology", "28":"Department of Chemical Engineering", "30":
        "National Space Institute", "31":"Department of Electrical Engineering", "33": "Department of Micro and Nanotechnology",
        "34": "Department of Photonics Engineering","41": "Department of Mechanical Engineering","42":
         "DTU Management Engineering","46": "DTU Wind Energy", "47": "DTU", "83": "Other courses", "IHK":"DTU Diplom"}

for bros in depts:
    
    columns = ["COURSE NUMBER",
               "COURSE NAME",
               "LANGUAGE OF INSTRUCTION",
               "CREDIT LOAD",
               "YEAR",
               "SCHEDULE",
               "COURSE TYPE",
               "DEPARTMENT",
               "GENERAL COURSE OBJECTIVES",
               "LEARNING OBJECTIVES",
               "CONTENT",
               "RECOMMENDED PREREQUISITES",
               "COURSE RESPONSIBLE",
               ]
    index = np.array(np.arange(1,1001))
    df = pd.DataFrame(index=index, columns=columns)
    df = df.fillna("NaNs")
    

    baseurl = "http://kurser.dtu.dk/search?CourseCode=&SearchKeyword=&SchedulePlacement=&Department=%s&CourseType=&TeachingLanguage=" %bros
    
    
    buks = getit(baseurl)
    
    buks = BeautifulSoup(buks,"lxml")
    b = list()
    fin = []
    
    
    for row in buks('table', {'class': 'table'})[1].tbody('tr'):
        b.append(list(row('td')))
        
    b = b[1:]
    
    for pops in b:
        if len(pops)>=4:
            fin.append(pops)
        else:
            break
            
        
    for ro in range(len(fin)):
        v = []
        fil = filter(None,fin[ro][1].get_text().split("\n"))#[3]#.split("-")[1]#.split("|")
        v.append(fil[0].split("-")[0])
        v.append(fil[0].split("-")[1])
        v.append(fil[1].split("|")[0])
        v.append(fil[1].split("|")[1])
        v.append(fil[1].split("|")[2])
        v.append(' '.join(fil[2:]))
        v.append(','.join(fin[ro][2].get_text().split()))
        v.append(depts[bros])
        
        
        
        urlsec = "http://kurser.dtu.dk/course/%s" %v[0]
        twobuks = getit(urlsec)
        twobuks = BeautifulSoup(twobuks,"lxml")
        gg = None
        hh = None
        ii = None
        tempor = []
        stanlo = twobuks.find_all("div",class_="box")
        
        
        for tf in range(len(list(stanlo[1]))):
            try:
                if list(stanlo[1])[tf].text=="Learning objectives":
                    gg = tf
            except:
                pass
        for ie in list(stanlo[1])[1:gg]:
            try:
                tempor.append(ie.get_text(' '))
            except:
                tempor.append(ie)
        v.append(filter(None,''.join(tempor)))
        
        tempor = []
        gg = list(stanlo[1])[gg+2:]
        for tr in range(len(gg)):
            try:
                if gg[tr].text=="Content":
                    hh = tr
            except:
                pass
            
        for eg in gg[0:hh]:
            try:
                tempor.append(eg.get_text(' '))
            except:
                tempor.append(eg)
        v.append(filter(None,''.join(tempor)))
        
        hh = gg[hh+1:]
        for li in range(len(hh)):
            try:
                if hh[li].name == 'div':
                    ii = li
                    break
            except:
                pass
        
        tempor = []

        for cj in hh[:ii]:
            try:
                tempor.append(cj.get_text(' '))
            except:
                tempor.append(cj)
        v.append(filter(None,''.join(tempor)))
        
        gg = None
        hh = None
        tempor = []
        
        stanlo = twobuks.find_all("table")
        gg = stanlo[1].find_all("tr")
        
        for i in range(len(gg)):
            if "Recommended prerequisites" in gg[i].get_text():
                hh = ' '.join([k.text for k in list(gg[i])[1:]])
                hh = re.sub(ur'[\W_]+', u' ', hh, flags=re.UNICODE)
                hh = [ij for ij in hh.split() if  len(ij) == 5]
                hh = ','.join(set([ip for ip in hh if ip.isalpha()==False]))
                
        v.append(hh)
        
        gg = stanlo[2].find_all("tr")
        
        for crew in range(0,len(gg[0].find_all("a")),2):
            it = gg[0].find_all("a")[crew].text
            tempor.append(it)
            
        v.append(','.join(tempor))
        
        
        #print v
        df.loc[ro+1] = np.array(v)
    #print df
    df = df.drop([i for i in range(len(fin)+1,1001)])
    df.to_csv(depts[bros]+u'.csv', sep="\t", encoding="utf-8")
